### Import libraries for Deep Learning

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using device: ', {device})

In [5]:
path = '/Users/fervargas/Developer/py/cnn/chest_xray/'

In [6]:
transform = transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [7]:
full_dataset = datasets.ImageFolder(path, transform=transform)

In [ ]:
len(full_dataset)

In [ ]:
TRAIN_SIZE =  int(len(full_dataset) * 0.8) +1
VAL_SIZE = int(len(full_dataset) * 0.1 )
TEST_SIZE = int(len(full_dataset) * 0.1)
BATCH_SIZE = 64

In [ ]:
print(TRAIN_SIZE, VAL_SIZE, TEST_SIZE)
print(TRAIN_SIZE + VAL_SIZE + TEST_SIZE)

In [ ]:
training_set, val_set, test_set = random_split(full_dataset, [TRAIN_SIZE, VAL_SIZE, TEST_SIZE])

In [ ]:
training_loader = DataLoader(dataset=training_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# categories = ['Basset Hound', 'Beagle', 'Pug']
categories = ['cat', 'dog']
def plot_figure(image):
    plt.imshow(image.permute(1,2,0))
    plt.show()

rnd_sample_idx = np.random.randint(TEST_SIZE)
print(f'La imagen muestreada representa un: {categories[test_loader.dataset[rnd_sample_idx][1]]}')
image = test_loader.dataset[rnd_sample_idx][0]
image = (image - image.min()) / (image.max() -image.min() )
plot_figure(image)

In [ ]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for (xi, yi) in loader:
            xi = xi.to(device=device, dtype = torch.float32)
            yi = yi.to(device=device, dtype = torch.long)
            scores = model(xi) # mb_size, 10
            _, pred = scores.max(dim=1) #pred shape (mb_size )
            num_correct += (pred == yi.squeeze()).sum() # pred shape (mb_size), yi shape (mb_size, 1)
            num_total += pred.size(0)

        return float(num_correct)/num_total     
            

In [ ]:
def train(model, optimiser, epochs=100):
#     def train(model, optimiser, scheduler = None, epochs=100):
    model = model.to(device=device)
    for epoch in range(epochs):
        for (xi, yi) in training_loader:
            model.train()
            xi = xi.to(device=device, dtype=torch.float32)
            yi = yi.to(device=device, dtype=torch.long)
            scores = model(xi)

            cost = F.cross_entropy(input= scores, target=yi.squeeze())
#             cost = F.bin
        
            optimiser.zero_grad()           
            cost.backward()
            optimiser.step()           
            
        acc = accuracy(model, val_loader)
        if epoch%1 == 0:     
            print(f'Epoch: {epoch}, costo: {cost.item()}, accuracy: {acc},')
#         scheduler.step()
    

In [ ]:
model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,
                 padding=1, stride=1, bias=True),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= (2, 2), stride=(2,2)),
    
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,
                 padding=1, stride=1, bias=True),
        nn.BatchNorm2d(num_features=64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= (2, 2), stride=(2,2)),
    
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,
                 padding=1, stride=1, bias=True),
        nn.BatchNorm2d(num_features=128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= (2, 2), stride=(2,2)),
    
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,
                 padding=1, stride=1, bias=True),
        nn.BatchNorm2d(num_features=128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size= (2, 2), stride=(2,2)),
    
        nn.Flatten(),
        nn.Linear(in_features = 14*14*128, out_features = 2, bias=True)
        )

In [ ]:
epochs = 30
num_classes = 2
# lr = 0.000031
lr = 0.001
optimizer = optim.Adam(params = model.parameters(), lr= lr)
train(model, optimizer, epochs)
acc = accuracy(model, val_loader)


In [ ]:
print(acc)

In [ ]:
accuracy(model, test_loader)

In [ ]:
sum_y = 0
for i, (x, y) in enumerate(test_loader):
    print(i, x.shape, y)
    sum_y += sum(y)
print(sum_y)